This notebook illustrates the workflow to calculate variogram of 

In [4]:
# Add python script files to the system env
import sys  
sys.path.insert(1, "..\\My Functions")
# 
import numpy as np
import pandas as pd
import rasterio as rio
# Import the custom python script files
import my_Variogram
import my_RomanMetrics

In [5]:
# \\DISC
cwd = "..\\..\\"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [6]:
site_Name = "BASP"

In [7]:
# Set g
g = 4.66

In [8]:
# Set internal distance
distances_Int = [30,30,100,300,300]
# Set external distance
distances_Ext = [100,300,300,900,1200]

In [9]:
# Get the unique distances so we can calculate the variogram first
list_distances = list(set(distances_Int + distances_Ext))
list_distances.sort()
list_distances

[30, 100, 300, 900, 1200]

In [10]:
list_std = []
list_cv = []
list_range = []
list_nugget = []
list_sill = []
list_bins = []
list_experimental_model = []
list_fitted_model = []
for i in range(len(list_distances)):
    img = rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv " + str(list_distances[i]) + "m.tif")
    band = img.read(1).reshape(-1)
    temp_std = my_Variogram.calc_STD(band)
    temp_cv = my_Variogram.calc_CV(band)
    coordinates = my_Variogram.get_Coordinates_from_Rio(img)
    temp_range, temp_nugget, temp_sill, temp_bins, temp_experimental_model, temp_fitted_model = my_Variogram.calc_Variogram_Parameters(band,coordinates,list_distances[i])
    list_std.append(temp_std)
    list_cv.append(temp_cv)
    list_range.append(temp_range)
    list_nugget.append(temp_nugget)
    list_sill.append(temp_sill)
    list_bins.append(temp_bins)
    list_experimental_model.append(temp_experimental_model)
    list_fitted_model.append(temp_fitted_model)
    print(f"Distance {list_distances[i]} has been calculated!")

c:\ProgramData\anaconda3\envs\DISC\Lib\site-packages\skgstat\Variogram.py:1715: OptimizeWarning: Covariance of the parameters could not be estimated
  self.cof, self.cov = curve_fit(


Distance 30 has been calculated!
Distance 100 has been calculated!
Distance 300 has been calculated!
Distance 900 has been calculated!
Distance 1200 has been calculated!


In [11]:
# Make a dataframe to illustrate the numeric result of variograms
df_Vario = pd.DataFrame({
    "Distance": list_distances,
    "STD": list_std,
    "CV": list_cv,
    "Range": list_range,
    "Sill": list_sill,
    "Nugget": list_nugget
})
df_Vario

,Distance,STD,CV,Range,Sill,Nugget
0,30,0.107388,0.011019,20.000000,0.013728,6.829347e-03
1,100,0.505830,0.050378,70.000000,0.280141,2.311226e-12
2,300,2.249598,0.208518,210.000000,5.202114,1.558166e-01
3,900,7.137173,0.550165,179.047736,47.130686,1.619532e+00
4,1200,9.229855,0.722840,840.000000,89.699852,4.732420e+01


In [12]:
list_Rcv = []
list_RAW_Score = []
list_Rse = []
list_Rst = []
list_Rsv = []
list_ST_Score = []
for i in range(len(list_distances)):
    dist_Int = distances_Int[i]
    dist_Ext = distances_Ext[i]
    index_Int = list_distances.index(dist_Int)
    index_Ext = list_distances.index(dist_Ext)
    range_Int, nugget_Int, sill_Int, bins_Int, experimental_model_Int, fitted_model_Int = list_range[index_Int], list_nugget[index_Int], list_sill[index_Int], list_bins[index_Int], list_experimental_model[index_Int], list_fitted_model[index_Int]
    range_Ext, nugget_Ext, sill_Ext, bins_Ext, experimental_model_Ext, fitted_model_Ext = list_range[index_Ext], list_nugget[index_Ext], list_sill[index_Ext], list_bins[index_Ext], list_experimental_model[index_Ext], list_fitted_model[index_Ext]
    cv_Int = list_cv[index_Int]
    cv_Ext = list_cv[index_Ext]
    semivar_range_Exp_Int = my_Variogram.get_Semivar_Exp(dist_Int, bins_Int, experimental_model_Int)
    semivar_range_Exp_Ext = my_Variogram.get_Semivar_Exp(dist_Ext, bins_Ext, experimental_model_Ext)
    # Calculate Rcv
    Rcv = my_RomanMetrics.calc_Rcv(cv_Ext, cv_Int)
    # Calculate RAW Score
    RAW_Score = my_RomanMetrics.calc_RAWScore(Rcv)
    # Calculate Rse
    Rse = my_RomanMetrics.calc_Rse(g, range_Ext, range_Int)
    # Calculate Rst
    Rst = my_RomanMetrics.calc_Rst(my_RomanMetrics.calc_ST(semivar_range_Exp_Ext, nugget_Ext), my_RomanMetrics.calc_ST(semivar_range_Exp_Int, nugget_Int))
    # Calculate Rsv
    Rsv = my_RomanMetrics.calc_Rsv(my_RomanMetrics.calc_SV(fitted_model_Ext, range_Ext, nugget_Ext, sill_Ext), my_RomanMetrics.calc_SV(fitted_model_Int, range_Int, nugget_Int, sill_Int))
    # Calculate ST Score
    ST_Score = my_RomanMetrics.calc_STScore(Rcv, Rse, Rst, Rsv)
    list_Rcv.append(Rcv)
    list_RAW_Score.append(RAW_Score)
    list_Rse.append(Rse)
    list_Rst.append(Rst)
    list_Rsv.append(Rsv)
    list_ST_Score.append(ST_Score)
    print(f"Internal distance {dist_Int} and external distance {dist_Ext} has been computed!")


Internal distance 30 and external distance 100 has been computed!
Internal distance 30 and external distance 300 has been computed!
Internal distance 100 and external distance 300 has been computed!
Internal distance 300 and external distance 900 has been computed!
Internal distance 300 and external distance 1200 has been computed!


In [13]:
# Make a dataframe to illustrate the numeric result of roman metrics
df_Roman = pd.DataFrame({
    "Internal Distance": distances_Int,
    "External Distance": distances_Ext,
    "Rcv": list_Rcv,
    "Rse": list_Rse,
    "Rst": list_Rst,
    "Rsv": list_Rsv,
    "RAW Score": list_RAW_Score,
    "ST Score": list_ST_Score
})
df_Roman

,Internal Distance,External Distance,Rcv,Rse,Rst,Rsv,RAW Score,ST Score
0,30,100,3.572139,0.784802,0.990028,5.965000,0.139972,0.232891
1,30,300,17.924253,0.791319,0.936148,19.269141,0.027895,0.074068
2,100,300,3.139037,0.932233,-0.027075,1.910142,0.159285,0.381051
3,300,900,1.638458,0.966376,-0.000529,-0.151268,0.305165,0.639743
4,300,1200,2.466563,0.977386,-0.450296,0.948012,0.202711,0.441369
